In [1]:
import MetaTrader5 as mt5
from dotenv import load_dotenv
import os
import csv

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("EE_MT5_LOGIN"))
password = os.getenv("EE_MT5_PASSWORD")
server = os.getenv("EE_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# Fetch all symbols currently visible in Market Watch
symbols = mt5.symbols_get()

# Filter only active tradable symbols
active_symbols = [
    sym.name
    for sym in symbols
    if sym.visible and sym.trade_mode == mt5.SYMBOL_TRADE_MODE_FULL
]

# Print active symbols
print("Active symbols in Market Watch:")
for name in active_symbols:
    print(name)

# -----------------------------
# Export to CSV
# -----------------------------
csv_filename = "active_symbols.csv"

with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Symbol"])  # header
    for name in active_symbols:
        writer.writerow([name])

print(f"\nExported {len(active_symbols)} symbols to {csv_filename}")

# Shutdown MT5 connection
mt5.shutdown()

Active symbols in Market Watch:
EURUSD
AUDUSD
GBPUSD
NZDUSD
USDCAD
USDCHF
USDJPY
DE30
FRA40
JPN225
SPX500
UK100
US100
US30
XAGUSD
XAUUSD
WTI

Exported 17 symbols to active_symbols.csv


True

### Max Sharpe Weights

In [2]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from dotenv import load_dotenv
import os

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("EE_MT5_LOGIN"))
password = os.getenv("EE_MT5_PASSWORD")
server = os.getenv("EE_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# -----------------------------
# Config
# -----------------------------
risk_free_rate = 0.0  # annual risk-free rate

# FX pairs needed to convert index returns to USD
fx_map = {
    "GER30.pro": "EURUSD.pro",
    "JPN225.pro": "USDJPY.pro",   # flip sign
    "HK50.pro": "USDHKD.pro",     # flip sign
    "UK100.pro": "GBPUSD.pro",
    "EUSTX50.pro": "EURUSD.pro"
}

# -----------------------------
# Helpers
# -----------------------------
def fetch_mt5_data(symbol, days=60):
    utc_to = pd.Timestamp.now()
    utc_from = utc_to - pd.Timedelta(days=days)

    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, utc_from, utc_to)
    if rates is None:
        raise ValueError(f"Failed to fetch data for {symbol}")

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)
    return df


def calculate_log_returns(prices: pd.Series) -> pd.Series:
    return np.log(prices / prices.shift(1)).dropna()

# -----------------------------
# Load expected returns
# -----------------------------
# ai_factor_signal_acg.csv:  asset \t factor_signal
expected_returns_df = pd.read_csv("ai_factor_signal_ee.csv", sep="\t", header=None)
expected_returns_df.columns = ["asset", "factor_signal"]

# -----------------------------
# Z-score the factor signals
# -----------------------------
mean_signal = expected_returns_df["factor_signal"].mean()
std_signal = expected_returns_df["factor_signal"].std()

expected_returns_df["zscore_signal"] = (expected_returns_df["factor_signal"] - mean_signal) / std_signal

# Convert to dict
expected_returns_dict = dict(zip(expected_returns_df["asset"], expected_returns_df["zscore_signal"]))

symbols = list(expected_returns_dict.keys())

# -----------------------------
# Fetch price data for assets
# -----------------------------
data = {symbol: fetch_mt5_data(symbol) for symbol in symbols}

# -----------------------------
# Fetch FX data and compute FX log returns
# -----------------------------
fx_symbols = set(fx_map[s] for s in symbols if s in fx_map)
fx_data = {fx: fetch_mt5_data(fx) for fx in fx_symbols}

fx_returns = {}
for fx, df_fx in fx_data.items():
    lr = calculate_log_returns(df_fx["close"])

    # If pair is quoted as USDXXX (e.g., USDJPY, USDHKD), flip sign to get XXXUSD log return
    if fx.startswith("USD"):
        lr = -lr  # log(1/x) = -log(x)

    fx_returns[fx] = lr

# -----------------------------
# Compute USD-adjusted log returns for all assets
# -----------------------------
asset_returns = {}

for symbol in symbols:
    idx_lr = calculate_log_returns(data[symbol]["close"])

    if symbol in fx_map:
        fx_symbol = fx_map[symbol]
        fx_lr = fx_returns[fx_symbol]

        # Align index and FX by date
        combined = pd.concat([idx_lr, fx_lr], axis=1, join="inner")
        combined.columns = ["idx", "fx"]

        # USD-adjusted return: r_usd = r_index_local + r_fx
        asset_returns[symbol] = combined["idx"] + combined["fx"]
    else:
        # Already in USD terms
        asset_returns[symbol] = idx_lr

# -----------------------------
# Build aligned returns matrix
# -----------------------------
returns = pd.DataFrame(asset_returns).dropna(how="any")

# -----------------------------
# Covariance matrix (daily, log returns)
# -----------------------------
cov_matrix = returns.cov()

# Expected daily returns vector (assumed to be in USD terms already)
expected_returns = np.array([expected_returns_dict[symbol] for symbol in symbols])
expected_returns = expected_returns * returns.std().values  # scale by asset volatilities

# -----------------------------
# Max Sharpe optimization
# -----------------------------
def max_sharpe_ratio(expected_returns, cov_matrix, risk_free_rate=0.0):
    num_assets = len(expected_returns)

    def neg_sharpe(weights):
        port_ret = np.dot(weights, expected_returns)
        port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        return -(port_ret - risk_free_rate) / port_vol

    # Dollar-neutral: sum(weights) = 0
    constraints = [{"type": "eq", "fun": lambda w: np.sum(w)}]

    # Allow long/short
    bounds = [(-1, 1)] * num_assets

    # Start with zero‑sum initial guess
    init_guess = np.ones(num_assets)
    init_guess[: num_assets // 2] = 1
    init_guess[num_assets // 2 :] = -1
    init_guess = init_guess / np.sum(np.abs(init_guess))

    result = minimize(neg_sharpe, init_guess, bounds=bounds, constraints=constraints)

    if not result.success:
        raise ValueError(f"Optimization failed: {result.message}")

    weights = result.x
    daily_ret = np.dot(weights, expected_returns)
    daily_vol = np.sqrt(weights.T @ cov_matrix @ weights)

    return weights, daily_ret, daily_vol

# -----------------------------
# Run optimization
# -----------------------------
weights, daily_port_return, daily_port_vol = max_sharpe_ratio(
    expected_returns, cov_matrix, risk_free_rate=risk_free_rate
)

# -----------------------------
# Export optimal weights
# -----------------------------
weights_df = pd.DataFrame({
    "Symbol": symbols,
    "Weight": weights
})
weights_df.to_csv("ee_optimal_portfolio_weights.csv", header=False, index=False)
print("\nWeights exported to ee_optimal_portfolio_weights.csv")

# -----------------------------
# Annualized statistics (using daily log returns)
# -----------------------------
annual_port_return = (1 + daily_port_return)**252 - 1
annual_port_vol = daily_port_vol * np.sqrt(252)
annual_sharpe = (annual_port_return - risk_free_rate) / annual_port_vol

# -----------------------------
# Output
# -----------------------------
print("\nOptimal Portfolio Weights:")
for symbol, weight in zip(symbols, weights):
    print(f"{symbol}: {weight:.4f}")

print("\n--- Portfolio Statistics ---")
print(f"Daily Portfolio Return: {daily_port_return:.4%}")
print(f"Annualized Portfolio Return: {annual_port_return:.4%}")
print(f"Daily Portfolio Volatility: {daily_port_vol:.4%}")
print(f"Annualized Portfolio Volatility: {annual_port_vol:.4%}")
print(f"Annualized Sharpe Ratio: {annual_sharpe:.2f}")

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


Weights exported to ee_optimal_portfolio_weights.csv

Optimal Portfolio Weights:
AUDUSD: 0.0011
EURUSD: -0.7025
GBPUSD: -0.1529
NZDUSD: -0.0352
USDJPY: -0.1973
USDCHF: -0.2407
USDCAD: 0.9965
XAGUSD: 0.0762
XAUUSD: 0.0074
WTI: 0.1760
SPX500: -0.1649
US100: 0.3882
US30: 0.3515
DE30: 0.2081
JPN225: -0.3428
UK100: -0.5462
FRA40: 0.1774

--- Portfolio Statistics ---
Daily Portfolio Return: 1.4936%
Annualized Portfolio Return: 4093.3816%
Daily Portfolio Volatility: 0.3533%
Annualized Portfolio Volatility: 5.6088%
Annualized Sharpe Ratio: 729.81


True

### Volatility Targeted Weights using EWMA estimates

In [3]:
import pandas as pd
import numpy as np

# --- Load weights from comma-separated file ---
weights_df = pd.read_csv(
    "ee_optimal_portfolio_weights.csv",
    sep=",",
    header=None,
    engine="python",
    encoding="utf-8-sig"
)

weights_df.columns = ["asset", "weight"]

# Align weights to returns.columns order
weights_df = weights_df.set_index("asset").reindex(returns.columns)

# Extract weight vector
weights = weights_df["weight"].values

print("Loaded weights (aligned):")
print(weights)
print("\nAsset order:")
print(list(returns.columns))

# ============================================================
# --- EWMA COVARIANCE MATRIX ---
# ============================================================

lambda_ = 0.94  # RiskMetrics default
rets = returns.values
n = rets.shape[1]

# Start with sample covariance as initial value
ewma_cov = returns.cov().values.copy()

# Iterate through returns to update EWMA covariance
for t in range(1, len(rets)):
    r = rets[t-1].reshape(-1, 1)
    ewma_cov = lambda_ * ewma_cov + (1 - lambda_) * (r @ r.T)

# Portfolio EWMA volatility
daily_returns = returns @ weights
ewma_daily_vol = np.sqrt(weights.T @ ewma_cov @ weights)
ewma_annual_vol = ewma_daily_vol * np.sqrt(252)
ewma_annual_return = daily_returns.mean() * 252 
 
print("\n--- EWMA VOLATILITY FORECAST ---")
print(f"EWMA Daily Volatility: {ewma_daily_vol:0.2%}")
print(f"EWMA Annual Volatility: {ewma_annual_vol:0.2%}")
print(f"EWMA Daily Portfolio Return: {daily_returns.mean():0.2%}")
print(f"EWMA Annual Portfolio Return: {ewma_annual_return:0.2%}")

# ============================================================
# --- SCALE PORTFOLIO TO TARGET VOLATILITY USING EWMA ---
# ============================================================

target_vol = 0.10  # 10% annual volatility

scale_factor = target_vol / ewma_annual_vol
scaled_weights = weights * scale_factor

# Recompute volatility after scaling
scaled_daily_vol = np.sqrt(scaled_weights.T @ ewma_cov @ scaled_weights)
scaled_annual_vol = scaled_daily_vol * np.sqrt(252)

# Compute scaled returns
scaled_daily_returns = returns @ scaled_weights
scaled_avg_daily_return = scaled_daily_returns.mean()
scaled_annual_return = scaled_avg_daily_return * 252

print(f"\n--- SCALED PORTFOLIO (Target Vol = {target_vol:.0%}) ---")
print(f"Scale Factor: {scale_factor:.4f}")
print(f"Scaled Daily Volatility (EWMA): {scaled_daily_vol:0.2%}")
print(f"Scaled Annual Volatility (EWMA): {scaled_annual_vol:0.2%}")
print(f"Scaled Daily Portfolio Return: {scaled_avg_daily_return:0.2%}")
print(f"Scaled Annual Portfolio Return: {scaled_annual_return:0.2%}")

print("\nScaled Weights:")
for asset, w in zip(returns.columns, scaled_weights):
    print(f"{asset}: {w:.6f}")

# --- EXPORT SCALED WEIGHTS TO CSV ---
scaled_df = pd.DataFrame({
    "asset": returns.columns,
    "scaled_weight": scaled_weights
})

scaled_df.to_csv("ee_scaled_weights.csv", index=False)
print("\nScaled weights exported to ee_scaled_weights.csv")

Loaded weights (aligned):
[ 0.00113415 -0.70248129 -0.15293972 -0.03518197 -0.19729683 -0.24070195
  0.99654808  0.07615086  0.0074054   0.17600437 -0.16489281  0.38820168
  0.35147868  0.20807321 -0.34275616 -0.54617658  0.17743087]

Asset order:
['AUDUSD', 'EURUSD', 'GBPUSD', 'NZDUSD', 'USDJPY', 'USDCHF', 'USDCAD', 'XAGUSD', 'XAUUSD', 'WTI', 'SPX500', 'US100', 'US30', 'DE30', 'JPN225', 'UK100', 'FRA40']

--- EWMA VOLATILITY FORECAST ---
EWMA Daily Volatility: 0.38%
EWMA Annual Volatility: 5.96%
EWMA Daily Portfolio Return: 0.03%
EWMA Annual Portfolio Return: 7.75%

--- SCALED PORTFOLIO (Target Vol = 10%) ---
Scale Factor: 1.6774
Scaled Daily Volatility (EWMA): 0.63%
Scaled Annual Volatility (EWMA): 10.00%
Scaled Daily Portfolio Return: 0.05%
Scaled Annual Portfolio Return: 13.01%

Scaled Weights:
AUDUSD: 0.001902
EURUSD: -1.178310
GBPUSD: -0.256534
NZDUSD: -0.059013
USDJPY: -0.330937
USDCHF: -0.403743
USDCAD: 1.671564
XAGUSD: 0.127732
XAUUSD: 0.012421
WTI: 0.295222
SPX500: -0.276584


### Lot Sizing

In [4]:
import math
import pandas as pd
import MetaTrader5 as mt5
import numpy as np

# ============================
# USER CONFIG
# ============================
EQUITY = 99438.93  # set your account equity

# FX-exempt symbols (no price in formula)
FX_EXEMPT = ["USDJPY", "USDCHF", "USDCAD"]

# Global index → FX mapping
INDEX_FX_MAP = {
    "DE30": "EURUSD",
    "JPN225": "USDJPY",
    "UK100": "GBPUSD",
    "FRA40": "EURUSD"
}

# ============================
# MT5 INITIALIZATION
# ============================
mt5.initialize()

def get_latest_price(symbol):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return None
    return tick.ask

def fetch_prices(assets):
    return {a: get_latest_price(a) for a in assets}

def fetch_index_fx_rates():
    return {idx: get_latest_price(fx) for idx, fx in INDEX_FX_MAP.items()}

# ============================
# LOAD CSV FILES
# ============================
contract_df = pd.read_csv("ee_contract_size.csv")
weights_df = pd.read_csv("ee_scaled_weights.csv")

# Normalize column names
contract_df.columns = contract_df.columns.str.strip().str.lower()
weights_df.columns = weights_df.columns.str.strip().str.lower()

# Rename scaled_weight → weight
weights_df.rename(columns={"scaled_weight": "weight"}, inplace=True)

# Merge contract sizes + weights
df = contract_df.merge(weights_df, on="asset", how="left")

# ============================
# FETCH PRICES FROM MT5
# ============================
all_assets = df["asset"].tolist()
latest_prices = fetch_prices(all_assets)
index_fx_rates = fetch_index_fx_rates()

df["latest_price"] = df["asset"].map(latest_prices)

# ============================
# LOT SIZE CALCULATION
# ============================
def compute_lot(row):
    asset = row["asset"]
    weight = row["weight"]
    contract_size = row["contract_size"]
    price = row["latest_price"]

    # Missing weight or contract size
    if pd.isna(weight) or pd.isna(contract_size):
        return None

    # Contract size cannot be zero
    if contract_size == 0:
        return None

    # Rule 1: FX-exempt assets
    if asset in FX_EXEMPT:
        return (weight * EQUITY) / contract_size

    # Missing or zero price
    if price is None or price == 0:
        return None

    # Rule 2: Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        # Invert FX rate for JP225
        if asset in ["JPN225"]:
            fx_rate = 1 / fx_rate

        price = price * fx_rate

        # After conversion, price still cannot be zero
        if price == 0:
            return None

    # Final safety check
    denominator = price * contract_size
    if denominator == 0:
        return None

    return (weight * EQUITY) / denominator

from decimal import Decimal, ROUND_HALF_UP

df["target_lot_size"] = df.apply(compute_lot, axis=1)

df["target_lot_size"] = df["target_lot_size"].apply(
    lambda x: Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)
)

# ============================
# FETCH NET POSITIONS FROM MT5
# ============================

def fetch_net_positions():
    positions = mt5.positions_get()
    if positions is None:
        return {}

    net = {}

    for p in positions:
        symbol = p.symbol
        volume = p.volume if p.type == 0 else -p.volume  # BUY = +, SELL = -
        net[symbol] = net.get(symbol, 0) + volume

    return net

net_positions = fetch_net_positions()

df["current_holdings"] = pd.to_numeric(df["asset"].map(net_positions).fillna(0), errors="coerce")
df["target_lot_size"] = pd.to_numeric(df["target_lot_size"], errors="coerce")

df["difference"] = df["target_lot_size"] - df["current_holdings"]
df["difference"] = pd.to_numeric(df["difference"], errors="coerce").round(2)
# ============================
# OUTPUT
# ============================
print(df[["asset", "latest_price", "current_holdings", "target_lot_size", "difference"]].dropna())
df[["asset", "weight", "contract_size", "latest_price", "current_holdings", "target_lot_size"]]\
    .dropna()\
    .to_csv("ee_lot_sizes_output.csv", index=False)

# ============================
# GROSS TOTAL LOT SIZE
# ============================
df["abs_target_lot_size"] = df["target_lot_size"].abs()
df["current_holdings"] = df["current_holdings"].abs()
gross_target_lot_size = df["abs_target_lot_size"].sum()
gross_current_holdings = df["current_holdings"].sum()

print(f"\nGross Total Target Lot Size (absolute): {gross_target_lot_size:.2f}")
print(f"Gross Current Holdings (absolute): {gross_current_holdings:.2f}")

mt5.shutdown()

     asset  latest_price  current_holdings  target_lot_size  difference
0   AUDUSD       0.66827               0.0             0.00        0.00
1   EURUSD       1.16399               0.0            -1.01       -1.01
2   GBPUSD       1.34323               0.0            -0.19       -0.19
3   NZDUSD       0.57420               0.0            -0.10       -0.10
4   USDJPY     158.50500               0.0            -0.33       -0.33
5   USDCHF       0.79982               0.0            -0.40       -0.40
6   USDCAD       1.38905               0.0             1.66        1.66
7   XAGUSD      92.68400               0.0             0.03        0.03
8   XAUUSD    4617.65000               0.0             0.00        0.00
9      WTI      60.93000               0.0             4.82        4.82
10  SPX500    6926.88000               0.0            -3.97       -3.97
11   US100   25458.95000               0.0             2.54        2.54
12    US30   49118.00000               0.0             1.19     

True

In [5]:
print(df["target_lot_size"].dtype)

float64
